In [1]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.optimize

from datetime import datetime
import pytz

import gm2
import trfp
import helper_function_candidates as helper
import plotting_functions as plt2

Welcome to JupyROOT 6.12/04


In [2]:
run4141 = gm2.FixedProbe([4141], prefix='dev/FieldPlainRootOutput_')

In [3]:
time, freq, qual = run4141.getBasics()
time = time[1:,:]/1.0e9
freq = freq[1:,:]
qual = qual[1:,:]

qual[qual >= 2**16] -= 2**16
qual[qual >= 2**8] -= 2**8

freq2 = freq.copy()

freq2[qual > 0] = np.nan

Reading event 4360/4367 99.84%     
loop done: 100%                                    


In [ ]:
print np.reshape(qual[0,:],[378/3,3])

In [ ]:
print freq2.shape
print np.max(np.max(qual))

In [ ]:
dropped = np.empty(378)
for pr in range(378):
    dropped[pr] = float(np.count_nonzero(np.isnan(freq2[:,pr])))/float(freq2[:,pr].size)

    
print np.max(dropped)
plt.hist(dropped, bins = 50)

In [ ]:
times = np.arange(np.ceil(time[0, 377]),
                  np.floor(time[-1, 0]) + 1,
                  1)  # NOTE THIS IS 1 SECOND NOW, DOESN'T MATTER BECAUSE OF INTEGRATION METHOD

interp = np.empty((times.size, 378))
for pr in range(378):
    not_nan = ~np.isnan(freq2[:,pr])
    interp[:,pr] = np.interp(times,time[:,pr][not_nan],freq2[:,pr][not_nan])

In [ ]:
probe = 3

fig, axs = plt.subplots(4,1)

plt.sca(axs[0])
plt.plot(time[:,probe], freq[:,probe], '.')

# axs[0].set_xlim([1525283150,1525283350])

plt.sca(axs[1])
plt.plot(time[:,probe], qual[:,probe], '.')
# axs[1].set_xlim([1525283150,1525283350])

plt.sca(axs[2])
plt.plot(time[:,probe], freq2[:,probe], '.')

# axs[2].set_xlim([1525283150,1525283350])

plt.sca(axs[3])
plt.plot(times, interp[:,probe], '.')

fig.set_size_inches(12,12)
fig.tight_layout()

In [ ]:
# try filtering a super noisy probe
from scipy import signal
import allantools

probe = 100

taus, adev, _, _ = allantools.mdev(interp[:,probe], data_type='freq', rate=1, taus='all')

b, a = signal.butter(1,1./10.)
y = signal.filtfilt(b, a, interp[:,probe])

taus2, adev2, _, _ = allantools.mdev(y, data_type='freq', rate=1, taus='all')

fig, axs = plt.subplots(4,1)

plt.sca(axs[0])
plt.plot(times, interp[:,probe], '.')

# axs[0].set_xlim([1525283150,1525283350])

plt.sca(axs[1])
plt.loglog(taus, adev, '.')

plt.sca(axs[2])
plt.plot(times, y, '.')
# axs[1].set_xlim([1525283150,1525283350])

plt.sca(axs[3])
plt.loglog(taus2, adev2, '.')

fig.set_size_inches(12,12)
fig.tight_layout()

In [ ]:
start = 0 * 42  # steps of 42

fig, axs = plt.subplots(126,3)

for i in range(126):
    for j in range(3):
        plt.sca(axs[i,j])
        taus, adev, _, _ = allantools.mdev(interp[:,start + 3*i + j], data_type='freq', rate=1, taus='octave')
        plt.loglog(taus, adev, ':.', markersize=3)
        axs[i,j].set_title('pr'+ str(start + 3*i + j))

fig.set_size_inches(15,240)
fig.tight_layout()

# fig.savefig('all_allan_4141.png', dpi=100)

In [ ]:
pr = 100

print np.mean(interp2[:,pr] - interp[:,pr])
print np.std(interp2[:,pr] - interp[:,pr])

fig, ax = plt.subplots(1,1)
plt.plot(times, interp2[:,pr] - interp[:,pr], '.')

fig.set_size_inches(12,4)
fig.tight_layout()